# Equity Volatility Fundamentals

This notebook demonstrates how to access and use the functionalities of **ETI Volatility Surfaces** within the **LSEG Financial Analytics SDK**. 

ETI (Exchange Traded Instruments) volatility surfaces provide implied volatility data across different strikes and maturities for equity options. This enables quantitative analysis, risk management, and derivatives pricing for equity markets.

**You will be able to:**
- Configure and request volatility surface data
- Parametrize volatility models (SSVI, SVI)
- Visualize volatility surfaces with interactive plots
- Export results for further analysis

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.market_data.eq_volatility` - for ETI volatility surface construction

This notebook uses external libraries **pandas, plotly**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.market_data import eq_volatility as ev

import plotly.graph_objects as go
import pandas as pd
import datetime as dt
import json


## Data Preparation

Volatility surface preparation follows a structured 3-step process:

**Prerequisites:** 
- Valid LSEG credentials and market data entitlements
- Instrument must have active options market with sufficient data

**Process Overview:**
1. **Surface Definition** - Specify the underlying instrument (equity RIC)
2. **Surface Parameters** - Configure volatility model and calculation settings
3. **Request Creation** - Combine definition and parameters into a request object  

Each step builds upon the previous, creating a complete request specification for the volatility surface engine.

In [2]:
print("Step 1: Creating Surface Definition...")
# Select a RIC for equities and indices
ric = "AAPL.O@RIC"

# Create surface definition object
surface_definition = ev.EtiSurfaceDefinition(
        instrument_code = ric
        # exchange = 'NSQ'  # NASDAQ
        )
print(f"   ✓ Instrument: {surface_definition.instrument_code}")


print("Step 2: Configuring Surface Parameters...")
surface_parameters = ev.EtiSurfaceParameters(
        calculation_date = dt.datetime.strptime("2025-01-18", "%Y-%m-%d"),
        time_stamp = ev.CurvesAndSurfacesTimeStampEnum.DEFAULT,          # Options: CLOSE, OPEN, SETTLE, DEFAULT
        input_volatility_type = ev.InputVolatilityTypeEnum.IMPLIED,      # Options: IMPLIED, QUOTED
        volatility_model = ev.CurvesAndSurfacesVolatilityModelEnum.SSVI, # Options: SVI, SSVI
        moneyness_type = ev.MoneynessTypeEnum.SPOT,                      # Options: SPOT
        price_side = ev.CurvesAndSurfacesPriceSideEnum.MID,              # Options: BID, MID, ASK
        x_axis = ev.XAxisEnum.STRIKE,                                    # Options: DATE, DELTA, MONEYNESS, STRIKE, TENOR
        y_axis = ev.YAxisEnum.DATE                                       # Options: same as X-axis
    )
print(f"   ✓ Surface Parameters: {surface_parameters}")


print("Step 3: Create request item...")
# Create the main request object  with basic configuration
request_item = ev.EtiVolatilitySurfaceRequestItem(
        surface_tag = f"{ric}_Volsurface",
        underlying_definition = surface_definition,
        surface_parameters = surface_parameters,
        underlying_type = ev.CurvesAndSurfacesUnderlyingTypeEnum.ETI,
        surface_layout = ev.SurfaceOutput(
            format = ev.FormatEnum.MATRIX,  # Options: LIST, MATRIX
        )
    )
print(f"   ✓ Request Item: {request_item}")

Step 1: Creating Surface Definition...
   ✓ Instrument: AAPL.O@RIC
Step 2: Configuring Surface Parameters...
   ✓ Surface Parameters: {'calculationDate': '2025-01-18T00:00:00Z', 'timeStamp': 'Default', 'inputVolatilityType': 'Implied', 'volatilityModel': 'SSVI', 'moneynessType': 'Spot', 'priceSide': 'Mid', 'xAxis': 'Strike', 'yAxis': 'Date'}
Step 3: Create request item...
   ✓ Request Item: {'surfaceTag': 'AAPL.O@RIC_Volsurface', 'underlyingDefinition': {'instrumentCode': 'AAPL.O@RIC'}, 'surfaceParameters': {'calculationDate': '2025-01-18T00:00:00Z', 'timeStamp': 'Default', 'inputVolatilityType': 'Implied', 'volatilityModel': 'SSVI', 'moneynessType': 'Spot', 'priceSide': 'Mid', 'xAxis': 'Strike', 'yAxis': 'Date'}, 'underlyingType': 'Eti', 'surfaceLayout': {'format': 'Matrix'}}


## Request Execution

In [3]:
# Execute the calculation using the calculate function
# The 'universe' parameter accepts a list of request items for batch processing
try:
    response = ev.calculate(universe=[request_item])

    # Display response structure information
    surface_data = response['data'][0]
    if 'surface' in surface_data:
        print(f"   Calculation successful!")
        print(f"   Surface data points available: {len(surface_data['surface'])}")
    else:
        print("   No surface data found in response")
    
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise

   Calculation successful!
   Surface data points available: 20


## Results Display

Once the volatility surface calculation is complete, we can visualize the results using multiple chart types to gain different insights:

**Visualization Options:**

- **3D Surface Plot** - Interactive 3D visualization of the complete volatility surface
- **Volatility Smiles** - Traditional 2D plots showing volatility curves by expiration

The plotting functions automatically convert the API response into pandas DataFrames and generate interactive Plotly visualizations.

In [4]:
# Plotting utils

def get_vol_surface_df(response, index=0):
    """
    Extract and format volatility surface data from API response into a DataFrame.
    
    This function processes the raw volatility surface data from an API response,
    handles various error conditions, and returns a properly formatted DataFrame
    suitable for visualization functions.

    """
    try:
        vol_surface = response['data'][index]['surface']
    except KeyError:
        print("No surface data available in response.")
        return None
    except TypeError:
        print("No surface data available in response.")
        return None

    expiries = vol_surface[0][1:]
    strikes = []
    values = []

    for row in vol_surface[1:]:
        strikes.append(row[0])
        values.append(row[1:])

    strikes = [round(float(s), 2) if isinstance(s, (int, float)) else s for s in strikes]

    surface_df = pd.DataFrame(values, index=strikes, columns=expiries).T
    surface_df = surface_df.astype(float).round(2)

    return surface_df

def plot_volatility_surface_plot(surf_table, x_axis, y_axis, colorscale="Turbo"):
    """
    Create an interactive 3D surface plot of the volatility surface with contour lines.
    
    This function generates a three-dimensional visualization where the x-axis represents
    moneyness/strikes, y-axis represents expiries, and z-axis represents volatility values.
    Contour lines are added for better depth perception.

    Parameters
    ----------
    surf_table : pd.DataFrame
        A DataFrame representing the volatility surface with expiries as index
        and strike prices/moneyness as columns. Values should be volatility levels.
    colorscale : str or list, optional
        Plotly colorscale name (e.g., 'Viridis', 'Turbo', 'Plasma') or custom colorscale list.
        Default is "Turbo".

    Returns
    -------
    plotly.graph_objects.Figure
  
    """
    if len(surf_table) < 2:
        fig = go.Figure()
        fig.add_annotation(
            text="Not enough data to display 3D surface",
            xref="paper", yref="paper",
            x=0.5, y=0.5,
            showarrow=False,
            font=dict(size=18),
            align="center"
        )
        fig.update_layout(
            title=dict(
                text="3D Surface Plot of Volatility Surface",
                x=0.5,
                xanchor="center",
                yanchor="top",
                y=0.95,
                font=dict(size=16)
            ),
            autosize=True,
            height=450,
            margin={"l": 0, "r": 0, "b": 0, "t": 50},
            dragmode=False,
        )
        return fig

    x = surf_table.columns
    y = surf_table.index
    z = surf_table.values

    fig = go.Figure(data=[go.Surface(
        z=z, 
        x=x, 
        y=y, 
        colorscale=colorscale,
        showscale=False,
        contours={
            "y": {
                "show": True,
                "color": "black",
                "highlightcolor": "black",
                "size": 0.05
            },
            "z": {
                "show": True,
                "color": "black",
                "highlightcolor": "black",
                "size": 0.05
            }
        }
    )])

    fig.update_layout(
        title=dict(
            text="3D Surface Plot of Volatility Surface",
            x=0.5,
            xanchor="center",
            yanchor="top",
            y=0.95,
            font=dict(size=16)
        ),
        scene={
            "xaxis_title": x_axis,
            "yaxis_title": y_axis,
            "zaxis_title": "Volatility",
            "xaxis": {"showgrid": True},
            "yaxis": {"showgrid": True},
            "zaxis": {"showgrid": True},
            "camera": {
                "eye": {"x": 0.96, "y": -1.53, "z": 0.39},
                "center": {"x": 0.02, "y": -0.07, "z": -0.21},
                "up": {"x": -0.18, "y": 0.27, "z": 0.95},
                "projection": {"type": "perspective"}
            }
        },
        dragmode=False,
        autosize=True,
        height=450,
        width=800,
    )

    return fig

def plot_surface_smile_by_expiry(surf_table, x_axis, y_axis):
    """
    Create a 2D line plot showing volatility smiles for different expiry dates.
    
    This function plots multiple volatility smile curves, with each curve representing
    a different expiry date. The x-axis shows moneyness/strikes and the y-axis shows
    volatility levels.

    Parameters
    ----------
    surf_table : pd.DataFrame
        A DataFrame with expiries as index and strike prices/moneyness as columns.
        Values should be volatility levels for each expiry-strike combination.

    Returns
    -------
    plotly.graph_objects.Figure
        Interactive line plot with each expiry represented as a separate trace.
        
    """
    fig = go.Figure()

    for expiry in surf_table.index:
        fig.add_trace(go.Scatter(
            x=surf_table.columns.astype(float),
            y=surf_table.loc[expiry],
            mode='lines+markers',
            name=expiry
        ))

    fig.update_layout(
        title=dict(
            text='Surface Smile by Expiry',
            x=0.5,
            xanchor="center",
            yanchor="top",
            y=0.95,
            font=dict(size=16)
        ),
        xaxis_title=x_axis,
        yaxis_title='Volatility',
        legend_title=y_axis,
        template='plotly_white',
        dragmode=False,
        autosize=True,
        height=450,
        width=800,
    )

    return fig

In [5]:
# Convert API response to pandas DataFrame for easier manipulation 
# This function extracts volatility data points and organizes them by strike/expiry 
surface_df = get_vol_surface_df(response)

# Extract axis names for labeling plots
x_axis = surface_parameters.x_axis.name
y_axis = surface_parameters.y_axis.name

print("Surface DataFrame Info:") 
print(f"   ✓ Shape: {surface_df.shape} (rows × columns)") 
print(f"   ✓ x_axis: {x_axis}") 
print(f"   ✓ y_axis: {y_axis}") 

Surface DataFrame Info:
   ✓ Shape: (14, 19) (rows × columns)
   ✓ x_axis: STRIKE
   ✓ y_axis: DATE


In [6]:
surface_df

,114.99,137.988,160.986,172.485,183.984,195.483,206.982,218.481,224.2305,229.98,235.7295,241.479,252.978,264.477,275.976,287.475,298.974,321.972,344.97
2025-02-21,80.50,67.58,54.68,48.10,41.48,35.09,29.72,26.40,25.62,25.33,25.42,25.77,26.94,28.40,29.95,31.49,32.98,35.76,38.26
2025-03-21,67.06,56.38,45.95,40.80,35.82,31.25,27.48,24.89,24.08,23.57,23.32,23.27,23.64,24.38,25.32,26.35,27.41,29.48,31.42
2025-04-17,60.19,50.76,41.74,37.43,33.35,29.69,26.68,24.51,23.75,23.20,22.84,22.64,22.62,22.97,23.54,24.25,25.02,26.63,28.21
2025-05-16,55.41,46.93,38.99,35.27,31.83,28.79,26.29,24.41,23.71,23.16,22.76,22.48,22.25,22.34,22.65,23.11,23.66,24.90,26.18
2025-06-20,51.44,43.81,36.82,33.63,30.72,28.17,26.06,24.42,23.79,23.27,22.85,22.54,22.16,22.07,22.18,22.44,22.80,23.69,24.69
2025-07-18,49.09,42.00,35.61,32.73,30.13,27.86,25.97,24.48,23.88,23.38,22.97,22.65,22.22,22.03,22.03,22.17,22.42,23.10,23.93
2025-08-15,47.21,40.58,34.68,32.06,29.70,27.64,25.92,24.55,23.99,23.51,23.11,22.78,22.31,22.06,21.99,22.04,22.20,22.72,23.39
2025-09-19,45.33,39.17,33.78,31.41,29.30,27.45,25.90,24.64,24.12,23.67,23.28,22.95,22.46,22.16,22.02,21.99,22.06,22.41,22.93
2025-10-17,44.09,38.26,33.22,31.02,29.05,27.34,25.90,24.72,24.23,23.79,23.42,23.09,22.59,22.26,22.08,22.00,22.03,22.27,22.68
2025-12-19,41.90,36.69,32.26,30.36,28.66,27.19,25.93,24.89,24.44,24.05,23.69,23.38,22.88,22.52,22.27,22.13,22.06,22.12,22.36


### 3D Surface Plot

An interactive 3D visualization allows you to explore the volatility surface from different angles and understand the relationship between strikes, expiries, and volatility levels.

In [7]:
# Create interactive 3D surface plot using the same DataFrame
# The plot allows rotation, zoom, and hover to explore volatility patterns
plot_volatility_surface_plot(surface_df, x_axis, y_axis)

In [8]:
# Plot individual volatility smiles for each expiration date
# Each line represents one expiry, showing how volatility changes with strike
# Useful for analyzing:
# - Volatility skew (asymmetry around ATM)  
# - Term structure (how smiles change over time)
# - ATM volatility levels across expiries

plot_surface_smile_by_expiry(surface_df, x_axis, y_axis)

## Save and Export

After generating volatility surface data and visualizations, you can export the results for further analysis, reporting, or integration with other systems:

**Export Options:**
- **Excel Format** - Save the structured DataFrame for spreadsheet analysis and sharing
- **JSON Format** - Export the complete API response for integration with other applications

In [9]:
# Create filenames with timestamp
timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
excel_filename = f"volatility_surface_{timestamp}.xlsx"
json_filename = f"response_{timestamp}.json"

# Save DataFrame to Excel
surface_df.to_excel(excel_filename, index=False)

# Save response to JSON
with open(json_filename, 'w') as f:
    json.dump(dict(response), f, indent=2, default=str)

print(f"Files saved: {excel_filename}, {json_filename}")

Files saved: volatility_surface_20251203_124600.xlsx, response_20251203_124600.json


## Appendix: Complete Function Reference

### Available Classes and Functions in eq_volatility Module

The following is a complete list of all classes, enums, and functions available after importing from `lseg_analytics.pricing.market_data.eq_volatility`:

**Core Classes:**
- `EtiSurfaceDefinition` - Define underlying instrument specifications
- `EtiSurfaceParameters` - Configure volatility model parameters and settings
- `EtiVolatilitySurfaceRequestItem` - Main request object for surface calculation
- `SurfaceFilters` - Apply filters to surface data (staleness, etc.)
- `MoneynessWeight` - Configure weighting for different moneyness levels

**Response Classes:**
- `VolatilitySurfaceResponse` - Main response container
- `VolatilitySurfaceResponseItem` - Individual surface response
- `VolatilitySurfacePoint` - Individual volatility data point
- `SurfaceOutput` - Output configuration and formatting

**Configuration Enums:**
- `CurvesAndSurfacesVolatilityModelEnum` - Volatility models (SSVI, SABR, SVI, etc.)
- `InputVolatilityTypeEnum` - Input volatility types (IMPLIED, LOG_NORMAL, etc.)
- `MoneynessTypeEnum` - Moneyness calculation methods (SPOT, DELTA, STRIKE, etc.)
- `CurvesAndSurfacesPriceSideEnum` - Price side selection (BID, ASK, MID)
- `CurvesAndSurfacesTimeStampEnum` - Time stamp options (CLOSE, OPEN, DEFAULT, etc.)
- `XAxisEnum` & `YAxisEnum` - Surface layout axis configuration

**Filter Classes:**
- `MaturityFilter` - Filter by option maturity
- `StrikeFilter` & `StrikeFilterRange` - Filter by strike prices

**Main Functions:**
- `calculate()` - Execute volatility surface calculations
- `functions_all` - List all available functions